## Poisson Equation With Dirichlet BCs ##

Let $\phi = \phi(x,y)$ be a function of two Cartesian coordinates $x$ and $y$ such that:
\begin{equation}
\Delta \phi = \phi_{xx} + \phi_{yy} = 2x(y-1)(y-2x+xy+2)e^{x-y}, \enspace 0<x<1, \enspace 0<y<1 
\end{equation}
subject to zero Dirichlet boundary conditions.

This yields the following analytical solution:

\begin{equation}
\phi(x,y) = e^{x-y}x(1-x)y(1-y)
\end{equation}

In [1]:
from devito import *
from devito.petsc import PETScSolve, EssentialBC
import pandas as pd
from devito import configuration
import numpy as np
configuration['opt'] = 'noop'

import os
os.environ["PETSC_ARCH"] = "jan25"
os.environ["PETSC_DIR"] = "/home/zl561/petsc"

In [2]:
print(os.environ["PETSC_ARCH"])

jan25


In [3]:
# analytical1 = pd.read_csv('results/phi_analytical_11.csv', header=None)
# numerical1 = pd.read_csv('results/phi_11.csv', header=None)

In [4]:
# Subdomains to implement BCs
class SubTop(SubDomain):
    name = 'subtop'
    def define(self, dimensions):
        x, y = dimensions
        return {x: x, y: ('right', 1)}
sub1 = SubTop()

class SubBottom(SubDomain):
    name = 'subbottom'
    def define(self, dimensions):
        x, y = dimensions
        return {x: x, y: ('left', 1)}
sub2 = SubBottom()

class SubLeft(SubDomain):
    name = 'subleft'
    def define(self, dimensions):
        x, y = dimensions
        return {x: ('left', 1), y: y}
sub3 = SubLeft()

class SubRight(SubDomain):
    name = 'subright'
    def define(self, dimensions):
        x, y = dimensions
        return {x: ('right', 1), y: y}
sub4 = SubRight()

In [5]:
def analytical(x, y):
    return 2.0*x*(y-1.0)*(y - 2.0*x + x*y + 2.0) * np.exp(x-y)


Lx = np.float64(1.)
Ly = np.float64(1.)

n_values = [11]
errors = []

In [6]:
# for n in n_values:
n =11
grid = Grid(shape=(n, n), extent=(Lx, Ly), subdomains=(sub1,sub2,sub3,sub4,), dtype=np.float64)

phi = Function(name='phi', grid=grid, space_order=2, dtype=np.float64)

rhs = Function(name='rhs', grid=grid, space_order=2, dtype=np.float64)

eqn = Eq(rhs, phi.laplace, subdomain=grid.interior)

# Initial guess - satisfies BCs
phi.data[:] = np.float64(0.)

tmpx = np.linspace(0, Lx, n).astype(np.float64)
tmpy = np.linspace(0, Ly, n).astype(np.float64)
Y, X = np.meshgrid(tmpx, tmpy)
rhs.data[:] = np.float64(2.0*X*(Y-1.0)*(Y - 2.0*X + X*Y + 2.0)) * np.float64(np.exp(X-Y))

# # Create boundary condition expressions using subdomains
x, y = grid.dimensions

bcs = [EssentialBC(phi, 0., subdomain=sub1)]
bcs += [EssentialBC(phi, 0., subdomain=sub2)]
bcs += [EssentialBC(phi, 0., subdomain=sub3)]
bcs += [EssentialBC(phi, 0., subdomain=sub4)]

petsc = PETScSolve([eqn]+bcs, target=phi)

with switchconfig(openmp=False):
    op = Operator(petsc, opt='noop')

op.apply()

phi_analytical = analytical(X, Y)

error = np.amax(np.abs(phi.data[1:-1,1:-1]-phi_analytical[1:-1,1:-1]))

errors.append(error)

    # pd.DataFrame(phi.data[:]).to_csv(f"results/phi_{n}.csv", header=None, index=None)

Trying to access deprecated config `openmp`. Using `language` instead
Trying to access deprecated config `openmp`. Using `language` instead
Trying to access deprecated config `openmp`. Using `language` instead
/tmp/devito-jitcache-uid1562842/51bf64ef4960ac114d38261f299a453e42dd15b1.c:8:10: fatal error: petscsnes.h: No such file or directory
    8 | #include "petscsnes.h"
      |          ^~~~~~~~~~~~~
compilation terminated.
FAILED compiler invocation: gcc -march=native -O3 -g -fPIC -Wall -std=c99 -Wno-unused-result -Wno-unused-variable -Wno-unused-but-set-variable -ffast-math -shared -fopenmp -Wl,-rpath,/home/zl561/petsc/jan25/lib -I/home/zl561/petsc/include -I/home/zl561/petsc/jan25/include /tmp/devito-jitcache-uid1562842/51bf64ef4960ac114d38261f299a453e42dd15b1.c -L/home/zl561/petsc/jan25/lib -lm -lpetsc -o /tmp/devito-jitcache-uid1562842/51bf64ef4960ac114d38261f299a453e42dd15b1.so


CompileError: module compilation failed